# Average Rating YoY

In [ ]:
# Avg rating viz function, YoY increments if existing

In [ ]:
# Execution (2024)

# Rating Frequency

# Weekly Rating Distribution

# Emotion Frequency

# Activity Frequency

# Activity-Emotion Correlation

# Location-Emotion Correlation

# Sick Days 🤒

# Productivity

# Weighted Avg Rating per Tag

# Locations Visited 🛫

# Word Clouds ☁️